In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import textwrap
import helper_400
import numpy as np
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
import libpysal
import spreg
from libpysal.weights import Queen

%load_ext autoreload
%autoreload 2
helper_400.set_sns_style()

import plotting

hs_dict = plotting.get_hotspopt_dict()

<Figure size 800x600 with 0 Axes>

## Spatial Auto-Poisson Regression

Implementation of Spatial Auto-Poisson Regression from: https://watermark.silverchair.com/forestscience0061.pdf

In [2]:
gdf = gpd.read_file("outputs/d2-events-2d-230929_SVI_shapefile.geojson")

In [3]:
gdf.replace(-999, np.nan, inplace=True) 
gdf.dropna(inplace=True)

In [14]:
# From the ref: "In this study, we define the nearest eight atlas blocks (Queen’s adjacency) as the neighbors, 
# and each neighboring block is equally weighted. Thus, the average response value of the nearest eight 
# neighboring blocks is used as the autocovariate in the AP model."

w = Queen.from_dataframe(gdf) 
w.transform = 'r' # row-standardize the weights

/var/folders/lc/q1l1y0k176b0h_m5yz0s_r5w0000gn/T/ipykernel_16815/1663077478.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)


('WARNING: ', 5157, ' is an island (no neighbors)')
('WARNING: ', 8128, ' is an island (no neighbors)')
('WARNING: ', 16711, ' is an island (no neighbors)')
('WARNING: ', 17171, ' is an island (no neighbors)')
('WARNING: ', 17731, ' is an island (no neighbors)')
('WARNING: ', 17732, ' is an island (no neighbors)')


/Users/anatrisovic/Projects/coexposure/.venv/lib/python3.12/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 13 disconnected components.
 There are 6 islands with ids: 5157, 8128, 16711, 17171, 17731, 17732.
  warnings.warn(message)


In [5]:
def compute_autocovariate(gdf, response_values):
    """
    Compute autocovariate for a GeoDataFrame using Queen contiguity.
    gdf: GeoDataFrame.
    response_values: array-like of response values for each observation in the gdf.
    Returns an array of autocovariate values.
    """
    autocovariates = []

    for i, neighbors in w.neighbors.items():
        response_neighborhood = [response_values[j] for j in neighbors]
        weights_neighborhood = w.weights[i]

        numerator = np.sum(np.array(weights_neighborhood) * np.array(response_neighborhood))
        denominator = np.sum(weights_neighborhood)

        if denominator == 0:
            autocovariate = 0
        else:
            autocovariate = numerator / denominator

        autocovariates.append(autocovariate)

    return np.array(autocovariates)

## Example

In [6]:
outcome_ = ['EP_POV150',
 'EP_AGE65',
 'EP_NOVEH',
 'EP_AFAM',
 'EP_HISP',
 'EP_ASIAN',
 'EP_AIAN']
names_dict = {
    "hw":"EH-WF",
    "ws":"WF-WS",
    "hs":"EH-WS",
    "hws":"EH-WF-WS"
}

In [7]:
# Calculate the autocovariate values
y = gdf["hw"].values
B = compute_autocovariate(gdf, y)  

# Set up your predictors
X = pd.DataFrame()
X[outcome_] = gdf[outcome_] 
X['autocovariate'] = B  # Add the autocovariate as a new predictor
X = sm.add_constant(X)

# Fit the model
poisson_model = sm.GLM(y, X, family=sm.families.Poisson())
results1 = poisson_model.fit(cov_type='HC0') # Robust standard errors

In [8]:
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                17963
Model:                            GLM   Df Residuals:                    17954
Model Family:                 Poisson   Df Model:                            8
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -7542.7
Date:                Tue, 31 Oct 2023   Deviance:                       12568.
Time:                        13:45:24   Pearson chi2:                 4.08e+07
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3965
Covariance Type:                  HC0                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -1.6910      0.149    -11.363      0.000      -1.983      -1.399
EP_POV150         0.0244      0.005      5.175      0.000       0.015       0.034
EP_AGE65          0.0184      0.003      6.110      0.000       0.012       0.024
EP_NOVEH         -0.0897      0.020     -4.566      0.000      -0.128      -0.051
EP_AFAM          -0.0900      0.025     -3.631      0.000      -0.139      -0.041
EP_HISP          -0.0116      0.003     -4.400      0.000      -0.017      -0.006
EP_ASIAN         -0.1241      0.017     -7.330      0.000      -0.157      -0.091
EP_AIAN           0.0076      0.003      2.347      0.019       0.001       0.014
autocovariate     0.3141      0.017     18.116      0.000       0.280       0.348
=================================================================================
"""

## Loop for all univariate and multivariate regressions for all hotspots 

In [16]:
def glm_model(gdf, outcome, var, autocovariate=True, univariate=True, model=sm.families.Poisson()):
    """
    Fit a Poisson GLM to a GeoDataFrame.
    gdf: GeoDataFrame.
    outcome: string name of the outcome variable in gdf.
    autocovariate: use autopoisson or not.
    """
    y = gdf[outcome].values
    B = compute_autocovariate(gdf, y)

    X = pd.DataFrame()
    X[var] = gdf[var] 
    if autocovariate:
        X['autocovariate'] = B
    X = sm.add_constant(X)

    poisson_model = sm.GLM(y, X, family=model)
    results = poisson_model.fit(cov_type='HC0') 
    tab = summary_col(results, stars=True, float_format='%0.3f').tables[0]
    
    if autocovariate:
        if univariate:
            return tab.iloc[2:4] # exclude const and autocovariate
        else:
            return tab.iloc[:-2] # exclude autocovariate coef
    else:
        if univariate:
            return tab.iloc[2:] # exclude const
        else:
            return tab


def get_summary_tables(gdf, autocovariate=True, model=sm.families.Poisson()):
    results = []

    for i in ['hw', 'ws', 'hs','hws']:  
        name = names_dict[i]

        # Univariate
        res_list = []
        for var in outcome_:
            res = glm_model(gdf, i, var, autocovariate=autocovariate, model=model)
            res_list.append(res)
        
        # Combine univariate results
        univariate_res = pd.concat(res_list).reset_index()
        univariate_res.columns = ['Variable' + name, 'Univariate ' + name]
        results.append(univariate_res)

        # Multivariate 
        multivariate_res = glm_model(gdf, i, outcome_, autocovariate=autocovariate, model=model, univariate=False)
        multivariate_res.columns = ['Multivariate ' + name]

        # Combine univariate and multivariate results
        res1 = pd.concat(
            [multivariate_res.iloc[2:], multivariate_res.iloc[0:2]]).reset_index(drop=True)
        results.append(res1)

    result = pd.concat(results, axis=1)
    result = result.rename(columns={result.columns[0]: 'Index'})
    result.drop(columns=result.columns[[3, 6, 9]], inplace=True)

    return result

## Spatial Poisson

In [13]:
result = get_summary_tables(gdf, autocovariate=True, model=sm.families.Poisson())
result

,Index,Univariate EH-WF,Multivariate EH-WF,Univariate WF-WS,Multivariate WF-WS,Univariate EH-WS,Multivariate EH-WS,Univariate EH-WF-WS,Multivariate EH-WF-WS
0,EP_POV150,0.009**,0.024***,0.012***,0.029***,0.000***,0.000,-0.001,0.019***
1,,(0.004),(0.005),(0.003),(0.004),(0.000),(0.000),(0.005),(0.005)
2,EP_AGE65,0.031***,0.018***,0.036***,0.022***,-0.001***,0.000,0.031***,0.019***
3,,(0.003),(0.003),(0.002),(0.002),(0.000),(0.000),(0.005),(0.003)
4,EP_NOVEH,-0.073***,-0.090***,-0.072***,-0.088***,-0.001***,-0.002***,-0.083***,-0.100***
5,,(0.017),(0.020),(0.012),(0.014),(0.000),(0.000),(0.020),(0.021)
6,EP_AFAM,-0.230***,-0.090***,-0.270***,-0.099***,0.002***,0.002***,-0.257***,-0.099***
7,,(0.040),(0.025),(0.035),(0.023),(0.000),(0.000),(0.044),(0.029)
8,EP_HISP,-0.019***,-0.012***,-0.025***,-0.017***,0.001***,0.001***,-0.018***,-0.012***
9,,(0.003),(0.003),(0.003),(0.003),(0.000),(0.000),(0.003),(0.003)


# Non-spatial Poisson

In [11]:
result = get_summary_tables(gdf, autocovariate=False, model=sm.families.Poisson())
result

,Index,Univariate EH-WF,Multivariate EH-WF,Univariate WF-WS,Multivariate WF-WS,Univariate EH-WS,Multivariate EH-WS,Univariate EH-WF-WS,Multivariate EH-WF-WS
0,EP_POV150,0.018***,0.035***,0.014***,0.042***,0.004***,0.006***,0.016***,0.036***
1,,(0.003),(0.004),(0.002),(0.004),(0.000),(0.000),(0.003),(0.004)
2,EP_AGE65,0.038***,0.025***,0.042***,0.025***,-0.002***,-0.004***,0.040***,0.025***
3,,(0.002),(0.002),(0.002),(0.002),(0.000),(0.000),(0.002),(0.003)
4,EP_NOVEH,-0.064***,-0.079***,-0.095***,-0.099***,-0.004***,-0.005***,-0.076***,-0.092***
5,,(0.015),(0.018),(0.012),(0.013),(0.001),(0.001),(0.015),(0.017)
6,EP_AFAM,-0.317***,-0.123***,-0.397***,-0.133***,-0.007***,-0.007***,-0.369***,-0.129***
7,,(0.044),(0.028),(0.045),(0.028),(0.001),(0.000),(0.053),(0.033)
8,EP_HISP,-0.020***,-0.014***,-0.040***,-0.028***,-0.000,-0.002***,-0.029***,-0.019***
9,,(0.003),(0.002),(0.004),(0.002),(0.000),(0.000),(0.003),(0.003)
